In [1]:
import pandas as pd
import re
import yaml
from data_io import load_data_from_db, insert_collection_into_db
from merge import merge_by_key_column
from tokenize import tokenize, NAME
from io import BytesIO
import warnings
warnings.filterwarnings("ignore")

In [2]:
from collections import Counter
pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', None)

In [3]:
with open('params.yml', 'r') as f:
    params = yaml.full_load(f)

In [4]:
pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', None)

In [89]:
# df = load_data_from_db('SomeClasses', params)[0]

In [76]:
# df = df.astype('str')
# df['voltage_level'] = df['Техместо'].apply(lambda x: x.split('-')[2])
# # df[df['voltage_level'] == '12']
# df[(df['Название'].str.contains('БСК')) & (df['voltage_level'] == '12')]

In [74]:
# def extract_BSK(text):
#     text = text.split('_')
#     if text[-1] in ['BSK', '601', '602']:
#             return True
#     return None

# df['is_BSK'] = df['Номер класса'].apply(extract_BSK)
# df_BSK = df[df['is_BSK'] == True]
# df_BSK = df_BSK[df_BSK['voltage_level'] == '12']

In [75]:
# from pymongo import MongoClient

# connection = MongoClient(params['mongo_connection'])
# db_name = 'LinearShuntCompensator'
# collection_name = 'LinearShuntCompensator_supa'
# db = connection[db_name]
# db.drop_collection(collection_name)
# col = db[collection_name]
# col.insert_many(df_BSK.to_dict('records'))
# print(f'{collection_name} collection inserted into {db_name}')
# connection.close()

In [34]:
with open('params.yml', 'r') as f:
    params = yaml.full_load(f)

class_name = 'LinearShuntCompensator'

df_rs, df_supa, df_substations_matched = load_data_from_db(class_name, params)
df_supa = df_supa[df_supa['Техместо'] != 'PS110-000492-12-02-601']

In [35]:
for df in [df_rs, df_supa, df_substations_matched]:
    display(df.head(10))
    display(df.info())
    print('-'*10)

,_id,Uid,name,Класс,Substation_Uid
0,65aa7dcbbb547a0fa377cc18,77f06f45-fa82-4268-b4d6-d14a8c051610,БСК-110,Линейное шунтирующее компенсирующее устройство,31524575-2b37-4609-8a63-aefb5469e50c
1,65aa7dcbbb547a0fa377cc19,12c49a54-5ec2-464e-893d-15c4efc2ca6a,БСК-20,Линейное шунтирующее компенсирующее устройство,504dd67c-8cbd-417d-96df-4b68c79a7476
2,65aa7dcbbb547a0fa377cc1a,305230b0-61fd-4fb5-adac-bbee8dc03c0e,БСК-30,Линейное шунтирующее компенсирующее устройство,504dd67c-8cbd-417d-96df-4b68c79a7476
3,65aa7dcbbb547a0fa377cc1b,a5385f52-a917-488a-a5e2-cd9add89ced4,БСК-10,Линейное шунтирующее компенсирующее устройство,504dd67c-8cbd-417d-96df-4b68c79a7476


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   _id             4 non-null      object
 1   Uid             4 non-null      object
 2   name            4 non-null      object
 3   Класс           4 non-null      object
 4   Substation_Uid  4 non-null      object
dtypes: object(5)
memory usage: 292.0+ bytes


None

----------


,_id,Техместо,Диспетчерское наименование,Название,Местоположение,Номер класса,Верхнее ТМ,Район,voltage_level,is_BSK
1,65a794edf07a9af91d55cc65,PS110-000492-12-02-601-51,None,БСК,4210020700,PS_12_01_601_BSK,PS110-000492,None,12,True
2,65a794edf07a9af91d55da40,PS110-000505-12-02-602,None,Присоединение УКРМ 2,4220020100,PS_12_01_601,PS110-000505,None,12,True
3,65a794edf07a9af91d55e297,PS110-000532-12-01-601,None,Присоединение БСК-110,4230020100,PS_12_01_601,PS110-000532,None,12,True
4,65a794edf07a9af91d5618d0,PS110-000688-12-02-601,None,Присоединение УКРМ 1,5530020700,PS_12_01_601,PS110-000688,None,12,True


...

----------


,_id,Uid,Техместо,Наименование,Диспетчерское наименование,Класс
0,65a9950923f335544b44a09e,000fe64c-7b85-4bea-9ada-3e514d968485,PS110-000516,ПС 110 кВ Уропская,ПС 110 кВ Уропская,Substation
1,65a9950923f335544b44a09f,011f18d7-614e-4945-8bf8-81964a8c6a9f,PS110-001024,ПС 110 кВ Потаскуй,ПС 110 кВ Потаскуй,Substation
2,65a9950923f335544b44a0a0,01628ca3-8af1-4d26-84c3-52ee6c0f1adc,PS110-000253,ПС 110 кВ Пригородная,ПС 110 кВ Пригородная,Substation
3,65a9950923f335544b44a0a1,018c4cfb-1f78-41e9-8e40-001a41114299,PS110-000519,ПС 110 кВ Городская,ПС 110 кВ Городская,Substation
4,65a9950923f335544b44a0a2,0193398d-8d84-40d9-8520-2f8ea56bf5e9,PS110-000294,ПС 110 кВ Коробейниково,ПС 110 кВ Коробейниково,Substation
5,65a9950923f335544b44a0a3,01d27eab-6a9e-41ba-9614-a782a63b430b,PS110-000268,ПС 110 кВ Опорная,ПС 110 кВ Опорная,Substation
6,65a9950923f335544b44a0a4,02697fc5-c1c1-4b96-a808-a905916b8feb,PS110-000008,ПС 110 кВ Гусиноозерская,ПС 110 кВ Гусиноозерская,Substation
7,65a9950923f335544b44a0a5,02b970fd-165b-4da9-9807-c8f21ee9b221,PS110-000387,ПС 110 кВ Затонская,ПС 110 кВ Затонская,Substation
8,65a9950923f335544b44a0a6,0355728d-e712-4133-97e2-0c3cbe4ef79c,PS110-001242,ПС 110 кВ Полярная,None,Substation
9,65a9950923f335544b44a0a7,037bf15d-c151-4d3d-938a-076a5e1d25f2,PS110-000807,ПС 110 кВ Орловский ГОК,ПС 110 кВ Орловский ГОК,Substation


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 822 entries, 0 to 821
Data columns (total 6 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   _id                         822 non-null    object
 1   Uid                         822 non-null    object
 2   Техместо                    822 non-null    object
 3   Наименование                822 non-null    object
 4   Диспетчерское наименование  818 non-null    object
 5   Класс                       822 non-null    object
dtypes: object(6)
memory usage: 38.7+ KB


None

----------


# Берем Техместо из ПС (ранее сопоставленных). Кладем в RS20

In [36]:
techplace_substations_matched = set(df_substations_matched['Техместо'].tolist())
techplace_substations_supa = set(df_supa['Верхнее ТМ'].tolist())
print('Между сопоставленными ПС и СУПА PowerTransformer общих по Техместу ПС:', len(techplace_substations_matched & techplace_substations_supa))

uid_substations_matched = set(df_substations_matched['Uid'].tolist())
uid_substations_rs = set(df_rs['Substation_Uid'].tolist())
print('Между сопоставленными ПС и RS20 PowerTransformer общих по Uid ПС:', len(uid_substations_matched & uid_substations_rs))

Между сопоставленными ПС и СУПА PowerTransformer общих по Техместу ПС: 4
Между сопоставленными ПС и RS20 PowerTransformer общих по Uid ПС: 2


In [37]:
uid2techplace = dict(zip(df_substations_matched['Uid'], df_substations_matched['Техместо']))
test = []
for key in uid2techplace.keys():
    test.append(uid2techplace[key] == df_substations_matched[df_substations_matched['Uid'] == key]['Техместо'].values[0])

assert all(test) == True
df_rs['Техместо_ПС'] = df_rs['Substation_Uid'].apply(lambda x: uid2techplace.get(x))
print('None:', df_rs['Техместо_ПС'].isna().sum())
df_rs

None: 0


,_id,Uid,name,Класс,Substation_Uid,Техместо_ПС
0,65aa7dcbbb547a0fa377cc18,77f06f45-fa82-4268-b4d6-d14a8c051610,БСК-110,Линейное шунтирующее компенсирующее устройство,31524575-2b37-4609-8a63-aefb5469e50c,PS110-000492
1,65aa7dcbbb547a0fa377cc19,12c49a54-5ec2-464e-893d-15c4efc2ca6a,БСК-20,Линейное шунтирующее компенсирующее устройство,504dd67c-8cbd-417d-96df-4b68c79a7476,PS110-000532
2,65aa7dcbbb547a0fa377cc1a,305230b0-61fd-4fb5-adac-bbee8dc03c0e,БСК-30,Линейное шунтирующее компенсирующее устройство,504dd67c-8cbd-417d-96df-4b68c79a7476,PS110-000532
3,65aa7dcbbb547a0fa377cc1b,a5385f52-a917-488a-a5e2-cd9add89ced4,БСК-10,Линейное шунтирующее компенсирующее устройство,504dd67c-8cbd-417d-96df-4b68c79a7476,PS110-000532


In [38]:
def name_norm(text):
    text = text.lower()
    text = text.replace('присоединение', '')
    match = re.findall('бск', text)
    if match:
        return match[0]

df_rs['name_norm'] = df_rs['name'].apply(name_norm)
df_supa['name_norm'] = df_supa['Название'].apply(name_norm)

In [39]:
df_rs['name_norm_techplace'] = df_rs['name_norm'] +'_'+ df_rs['Техместо_ПС']
df_supa['name_norm_techplace'] = df_supa['name_norm'] +'_'+ df_supa['Верхнее ТМ']
df_matched = pd.merge(df_rs, df_supa, on='name_norm_techplace')

In [40]:
df_matched

,_id_x,Uid,name,Класс,Substation_Uid,Техместо_ПС,name_norm_x,name_norm_techplace,_id_y,Техместо,Диспетчерское наименование,Название,Местоположение,Номер класса,Верхнее ТМ,Район,voltage_level,is_BSK,name_norm_y
0,65aa7dcbbb547a0fa377cc18,77f06f45-fa82-4268-b4d6-d14a8c051610,БСК-110,Линейное шунтирующее компенсирующее устройство,31524575-2b37-4609-8a63-aefb5469e50c,PS110-000492,бск,бск_PS110-000492,65a794edf07a9af91d55cc65,PS110-000492-12-02-601-51,None,БСК,4210020700,PS_12_01_601_BSK,PS110-000492,None,12,True,бск
1,65aa7dcbbb547a0fa377cc19,12c49a54-5ec2-464e-893d-15c4efc2ca6a,БСК-20,Линейное шунтирующее компенсирующее устройство,504dd67c-8cbd-417d-96df-4b68c79a7476,PS110-000532,бск,бск_PS110-000532,65a794edf07a9af91d55e297,PS110-000532-12-01-601,None,Присоединение БСК-110,4230020100,PS_12_01_601,PS110-000532,None,12,True,бск
2,65aa7dcbbb547a0fa377cc1a,305230b0-61fd-4fb5-adac-bbee8dc03c0e,БСК-30,Линейное шунтирующее компенсирующее устройство,504dd67c-8cbd-417d-96df-4b68c79a7476,PS110-000532,бск,бск_PS110-000532,65a794edf07a9af91d55e297,PS110-000532-12-01-601,None,Присоединение БСК-110,4230020100,PS_12_01_601,PS110-000532,None,12,True,бск
3,65aa7dcbbb547a0fa377cc1b,a5385f52-a917-488a-a5e2-cd9add89ced4,БСК-10,Линейное шунтирующее компенсирующее устройство,504dd67c-8cbd-417d-96df-4b68c79a7476,PS110-000532,бск,бск_PS110-000532,65a794edf07a9af91d55e297,PS110-000532-12-01-601,None,Присоединение БСК-110,4230020100,PS_12_01_601,PS110-000532,None,12,True,бск


In [41]:
print('Сопоставлено', (df_matched.shape[0])/(df_supa.shape[0]), 'СУПА')
print('Сопоставлено', (df_matched.shape[0])/(df_rs.shape[0]), 'RS')

Сопоставлено 1.0 СУПА
Сопоставлено 1.0 RS
